# 01 准确度的陷阱和混淆矩阵

## 分类准确度问题
一个癌症预测系统，输入体检信息，可以判断是否有癌症，如果一个系统的预测准确度是99.9%，而癌症的概率只有0.01%，那预测所有人都是健康的，可以达到99.99%的准确率，即这个预测系统是失败的

如此产生的原因是：对于极度偏斜(Skewed Data)的数据，只使用分类准确度是远远不够的

使用混淆矩阵做进一步的分析

## 混淆矩阵 (Confusion matrix)
对于二分类问题

0-Ngeative

1-Positive

xx:预测结果预测值


|真实值\\预测值|0|1|
|:----:|:----:|:----:|
|0|TN|FP|
|1|FN|TP|


# 02 精准率和召回率

在癌症预测系统中对于10000个病人进行分类，混淆矩阵如下：

|真实值\\预测值|0|1|
|:----:|:----:|:----:|
|0|9978|12|
|1|2|8|

精准率(预测为1的正确数在预测为1中的占比):$precision = \frac{TP}{TP+FP}$

精准率 = 8/(8+12) = 40%

召回率(预测为1的正确数在真实为1中的占比):$recall = \frac{TP}{TP+FN}$

召回率 = 8/(8+2) = 80%